# 4. Clustering dei Giocatori

**Obiettivo:** Raggruppare i giocatori in cluster omogenei basati sul loro stile di gioco. Questo è il cuore del mio progetto, dove cercherò di scoprire i "profili" statistici che definiscono i giocatori moderni.

1.  **Caricamento e Selezione Dati:** Carico il dataset arricchito e seleziono solo la stagione più rappresentativa di ogni giocatore.
2.  **Preparazione delle Feature:** Scelgo le statistiche più significative e le standardizzo.
3.  **Determinazione di 'k':** Uso l'Elbow Method per trovare il numero ottimale di cluster.
4.  **Addestramento e Valutazione:** Addestro il modello K-Means e valuto la qualità dei cluster con il Silhouette Score.
5.  **Salvataggio dei Risultati:** Salvo il DataFrame finale con l'ID del cluster per ogni giocatore.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import col, max, row_number, when
from pyspark.sql.window import Window

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.clustering.models import (
    assign_clusters, evaluate_clustering, 
    prepare_features_for_clustering, train_kmeans_model
)
from src.config.spark_config import SPARK_CONFIG
from src.utils.helpers import get_spark_session, save_dataframe

project_root = os.path.abspath(os.path.join('..'))
processed_data_dir = os.path.join(project_root, "data", "processed")
adv_metrics_path = os.path.join(processed_data_dir, "players_advanced_metrics.parquet")
output_path = os.path.join(processed_data_dir, "players_clustered.parquet")

spark = get_spark_session(
    app_name="NBA_Player_Clustering",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

### Fase 1: Caricamento Dati e Selezione dell'Ultima Stagione

Per analizzare lo stile "attuale" di un giocatore, ho bisogno di selezionare una singola stagione rappresentativa. Ho sviluppato una logica per prendere l'ultima stagione giocata, gestendo anche i casi di giocatori scambiati che hanno una riga 'TOT'.

In [ ]:
df_input = spark.read.parquet(adv_metrics_path)

window_spec_latest_season = Window.partitionBy("player").orderBy(col("season").desc(), col("g").desc())

df_with_rank = df_input.withColumn("rank", row_number().over(window_spec_latest_season))

df_latest_season_only = df_with_rank.filter(col("rank") == 1)

df_with_priority = df_latest_season_only.withColumn(
    "priority",
    when(col("tm") == "TOT", 1).otherwise(2)
)

final_window_spec = Window.partitionBy("player", "season").orderBy("priority", "g")
df_with_final_rank = df_with_priority.withColumn("final_rank", row_number().over(final_window_spec))

df_for_clustering_input = df_with_final_rank.filter(col("final_rank") == 1)

feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'
]

df_for_clustering = df_for_clustering_input.select(["player", "season"] + feature_cols).na.drop()
print(f"Numero di giocatori idonei per il clustering: {df_for_clustering.count()}")
df_for_clustering.show(5)

#### Interpretazione dell'output

L'output mi mostra quanti giocatori userò per il clustering e le sette feature che ho scelto. Questo sarà il mio input per l'algoritmo.

### Fase 2: Preparazione e Scaling delle Feature

K-Means è sensibile alla scala delle variabili, quindi devo standardizzarle. In questo modo, tutte le feature contribuiranno equamente al calcolo della distanza.

In [ ]:
df_prepared = prepare_features_for_clustering(df_for_clustering, feature_cols)

df_prepared.select("player", "features_scaled").show(5, truncate=False)

#### Interpretazione dell'output

La colonna `features_scaled` contiene i vettori con i valori standardizzati, pronti per essere usati dal modello.

### Fase 3: Determinazione del Numero Ottimale di Cluster (Elbow Method)

Una domanda fondamentale è: "quanti gruppi (k) devo creare?". L'Elbow Method mi aiuta a rispondere. Calcolo il costo per diversi valori di 'k' e cerco il "gomito" nel grafico, che rappresenta un buon compromesso.

In [ ]:
cost = []
k_range = range(2, 15)
print("Calcolo del costo WSSSE per diversi k...")

for k in k_range:
    kmeans = KMeans(featuresCol='features_scaled', k=k, seed=42)
    model = kmeans.fit(df_prepared)
    cost.append(model.summary.trainingCost)

plt.figure(figsize=(12, 6))
plt.plot(k_range, cost, 'bx-')
plt.xlabel('k (Numero di Cluster)')
plt.ylabel('Costo (WSSSE)')
plt.title('Elbow Method per la Scelta Ottimale di k')
plt.xticks(k_range)
plt.grid(True)
plt.show()

#### Analisi del Gomito

Il grafico mostra un "gomito" evidente intorno a `k=6`. Dopo questo punto, aggiungere altri cluster non porta grandi benefici. Quindi, ho scelto **`k=6`** perché mi dà un buon numero di profili mantenendo i cluster interpretabili.

### Fase 4: Addestramento del Modello Finale e Valutazione

Ora che ho scelto `k=6`, addestro il modello finale e assegno ogni giocatore a un cluster. Poi valuto la qualità del clustering con il Silhouette Score.

In [ ]:
k_optimal = 6
print(f"Addestramento del modello K-Means finale con k={k_optimal}...")

kmeans_model = train_kmeans_model(df_prepared, k=k_optimal)
predictions_df = assign_clusters(kmeans_model, df_prepared)

print("Esempi di giocatori con cluster assegnato:")
predictions_df.select("player", "season", "cluster_id").show(10)

silhouette_score = evaluate_clustering(predictions_df)
print(f"Punteggio Silhouette (validità del clustering): {silhouette_score:.3f}")

#### Interpretazione dell'output

La tabella mi mostra l'ID del cluster (da 0 a 5) per ogni giocatore. Il Silhouette Score è positivo, il che mi dice che i cluster sono ragionevolmente ben definiti. È un buon risultato per dati reali come questi.

### Fase 5: Salvataggio dei Risultati del Clustering

Salvo il DataFrame con le assegnazioni dei cluster. Questo file sarà l'input per l'ultimo notebook, dove interpreterò i profili.

In [ ]:
columns_to_save = ["player", "season"] + feature_cols + ["cluster_id"]
df_to_save = predictions_df.select(columns_to_save)

save_dataframe(df_to_save, output_path)

print(f"Risultati del clustering salvati con successo in '{output_path}'.")

### Conclusione e Prossimi Passi

Ho completato la parte di machine learning. Ho preparato i dati, scelto 'k', addestrato il modello e assegnato i cluster.

Nel prossimo notebook, `05_cluster_analysis_and_visualization.ipynb`, mi tufferò nell'analisi di questi cluster per capire cosa rappresentano.

In [ ]:
spark.stop()